# Análisis por máscara

In [1]:
from mvpa2.suite import *
datapath = '/media/sf_usr/share/data/objetos/data'

## Cargando los datos 

In [27]:
dhandle = OpenFMRIDataset(datapath) #creates a handle for the data
run_datasets = []
for run_id in [1,2,3,4]:
    # obtiene el path de la máscara

    mask_fname = os.path.join(datapath, 'sub001', 'masks', 'orig', 'L_AIPs.nii.gz')
    # Carga el diseño
    run_events = dhandle.get_bold_run_model(1, '001', run_id)
    # Carga la imagen funcional
    run_ds = dhandle.get_bold_run_dataset('001', 1, run_id, chunks=run_id -1, mask=mask_fname)
    # Carga los vectores
    run_ds.sa['targets'] = events2sample_attr(run_events, run_ds.sa.time_coords, noinfolabel='rest')
    # une los datos cargados en un solo array
    run_datasets.append(run_ds)
fds = vstack(run_datasets, a=0)

### Muestra lo que contienen los datos cargados

In [16]:
print fds.summary()

Dataset: 1000x1918@float32, <sa: chunks,run,subj,targets,task,time_coords,time_indices>, <fa: voxel_indices>, <a: imgaffine,imghdr,imgtype,mapper,voxel_dim,voxel_eldim>
stats: mean=10793 std=1504.07 var=2.26221e+06 min=389.71 max=17539.3

Counts of targets in each chunk:
  chunks\targets harsh lines rest soft
                  ---   ---   ---  ---
        0          27    27   169  27
        1          27    27   169  27
        2          27    27   169  27
        3          27    27   169  27

Summary for targets across chunks
  targets mean std min max #chunks
  harsh    27   0   27  27    4
  lines    27   0   27  27    4
   rest    169  0  169 169    4
   soft    27   0   27  27    4

Summary for chunks across targets
  chunks mean  std min max #targets
    0    62.5 61.5  27 169     4
    1    62.5 61.5  27 169     4
    2    62.5 61.5  27 169     4
    3    62.5 61.5  27 169     4
Sequence statistics for 1000 entries from set ['harsh', 'lines', 'rest', 'soft']
Counter-balance 

## Preprocesado

### Linear detrending

In [28]:
detrender = poly_detrend(fds, polyord=1, chunks_attr='chunks')

### Conversión a puntaje Z

In [29]:
zscore(fds, param_est=('targets', ['rest']))

### Remueve los volúmenes asignados a línea base

In [30]:
fds = fds[fds.sa.targets != 'rest']

### Promedia los volúmenes por cada corrida

In [31]:
run_averager = mean_group_sample(['targets', 'chunks'])
fds = fds.get_mapped(run_averager)
fds.sa.runtype = ['odd','even','odd','even','odd','even','odd','even','odd','even','odd','even']

In [39]:
type(fds.sa.chunks)

numpy.ndarray

## Clasificación

### Crea una instancia del clasificador

In [32]:
clf = LinearCSVMC()

### Hace una selección de los mejores vóxeles de acuerdo a un ANOVA



In [33]:
fsel = SensitivityBasedFeatureSelection(OneWayAnova(), FractionTailSelector(0.05, mode='select', tail='upper'))
fclf = FeatureSelectionClassifier(clf, fsel)

### Validación cruzada

In [40]:
cvte = CrossValidation(fclf, NFoldPartitioner(), errorfx=lambda p, t: np.mean(p == t), enable_ca=['stats'])

### Ejecución del análisis

In [41]:
cv_results = cvte(fds)

## Resultados

In [42]:
print cvte.ca.stats.as_string(description=True)

----------.
predictions\targets  harsh  lines  soft
            `------  -----  -----  ----- P' N' FP FN  PPV  NPV  TPR  SPC  FDR  MCC   F1
       harsh           1      2      0    3  8  2  3 0.33 0.62 0.25 0.71 0.67 -0.04 0.29
       lines           2      2      1    5  6  3  2  0.4 0.67  0.5 0.57  0.6  0.06 0.44
        soft           1      0      3    4  4  1  1 0.75 0.75 0.75 0.75 0.25  0.35 0.75
Per target:          -----  -----  -----
         P             4      4      4
         N             8      8      8
         TP            1      2      3
         TN            5      4      3
Summary \ Means:     -----  -----  -----  4  6  2  2 0.49 0.68  0.5 0.68 0.51  0.13 0.49
       CHI^2           6   p=0.65
        ACC           0.5
        ACC%          50
     # of sets         4    ACC(i) = 0.8-0.2*i p=0.23 r=-0.77 r^2=0.6

Statistics computed in 1-vs-rest fashion per each target.
Abbreviations (for details see http://en.wikipedia.org/wiki/ROC_curve):
 TP : true positive (

In [47]:
cv_results.samples

0.5
